In [170]:
import pandas as pd

# MIMIC IV

In [223]:
patient_df = pd.read_csv('/secure/shared_data/SOAP/MIMIC/PATIENTS.csv').drop(columns=['ROW_ID','DOD_HOSP', 'DOD_SSN'])
patient_df.columns, patient_df.shape

(Index(['SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'EXPIRE_FLAG'], dtype='object'),
 (46520, 5))

In [224]:
patient_df['DOB'] = pd.to_datetime(patient_df['DOB'])
patient_df['DOD'] = pd.to_datetime(patient_df['DOD'])

array(['F', 'M'], dtype=object)

In [250]:
admission_df = pd.read_csv('/secure/shared_data/SOAP/MIMIC/ADMISSIONS.csv')[admission_df['HOSPITAL_EXPIRE_FLAG']==1]
admission_df[['DISCHTIME', 'DEATHTIME', 'HOSPITAL_EXPIRE_FLAG']]

,DISCHTIME,DEATHTIME,HOSPITAL_EXPIRE_FLAG
9,2108-08-30 15:00:00,2108-08-30 15:00:00,1
57,2104-01-08 10:30:00,2104-01-08 10:30:00,1
63,2119-02-03 01:35:00,2119-02-03 01:35:00,1
70,2157-12-02 03:55:00,2157-12-02 03:55:00,1
88,2196-04-17 13:42:00,2196-04-17 13:42:00,1
...,...,...,...
58927,2147-04-14 10:00:00,2147-04-14 10:00:00,1
58957,2172-03-02 08:00:00,2172-03-02 08:00:00,1
58966,2112-09-17 08:20:00,2112-09-17 08:20:00,1
58970,2132-12-25 12:00:00,2132-12-25 12:00:00,1


In [246]:
admission_df = pd.read_csv('/secure/shared_data/SOAP/MIMIC/ADMISSIONS.csv')
# admission_df = admission_df[admission_df['DEATHTIME'].isna()]
admission_df = admission_df[admission_df['ADMISSION_TYPE']!= 'NEWBORN']
admission_df = admission_df.drop(columns=['ROW_ID', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'EDREGTIME', 'EDOUTTIME', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'])
admission_df.columns, admission_df.shape

(Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
        'ADMISSION_TYPE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
        'DIAGNOSIS'],
       dtype='object'),
 (51113, 11))

In [ ]:
admission_df['ADMITTIME'] = pd.to_datetime(admission_df['ADMITTIME'])
admission_df['DISCHTIME'] = pd.to_datetime(admission_df['DISCHTIME'])

admission_df = admission_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME']).reset_index(drop=True)
admission_df['NEXT_ADMITTIME'] = admission_df.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)

admission_df = admission_df.merge(
    patient_df[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'EXPIRE_FLAG']], 
    on='SUBJECT_ID', 
    how='left'
)

In [233]:
admission_df.shape

(45321, 14)

In [234]:
admission_df['AGE_AT_ADMISSION'] = admission_df.apply(
    lambda row: row['ADMITTIME'].year 
                - row['DOB'].year 
                - ((row['ADMITTIME'].month, row['ADMITTIME'].day) 
                   < (row['DOB'].month, row['DOB'].day)),
    axis=1
)

In [236]:
admission_df['SUBJECT_ID'] = admission_df['SUBJECT_ID'].astype(str)
admission_df['HADM_ID'] = admission_df['HADM_ID'].astype(str)

In [237]:
note_df = pd.read_csv(
    '/secure/shared_data/SOAP/MIMIC/NOTEEVENTS.csv',
    engine='python',  
    on_bad_lines='skip',     
    sep=',',                 
    dtype=str               
)
note_df.columns, note_df.shape

(Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
        'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT'],
       dtype='object'),
 (2083180, 11))

In [240]:
note_df = note_df[note_df['HADM_ID'].isin(admission_df['HADM_ID'])]
note_df.shape

(1202150, 11)

In [190]:
# note_counts = note_df.groupby(['SUBJECT_ID', 'HADM_ID']).size()

# multiple_notes = note_counts[note_counts > 1]

# print("Admissions with multiple notes:")
# print(multiple_notes)
# print(len(multiple_notes))

여러 노트가 존재하는 입원:
SUBJECT_ID  HADM_ID
10          184167      42
100         153952      10
10000       187813      16
10001       118420       2
10002       132138       3
                      ... 
99985       176670     134
99991       151118      36
99992       197084       7
99995       137810       6
99999       113369       9
Length: 52242, dtype: int64
총 입원 중 여러 노트가 있는 그룹의 수: 52242


In [243]:
expanded_admission_df = admission_df.merge(
    note_df[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'CATEGORY', 'DESCRIPTION', 'TEXT']],
    on=['SUBJECT_ID', 'HADM_ID'],
    how='left'
)
expanded_admission_df['CHARTDATE'] = pd.to_datetime(expanded_admission_df['CHARTDATE'], errors='coerce')
expanded_admission_df['CHARTTIME'] = pd.to_datetime(expanded_admission_df['CHARTTIME'], errors='coerce')

expanded_admission_df = expanded_admission_df.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME'])
expanded_admission_df = expanded_admission_df.reset_index(drop=True)
expanded_admission_df.head(10)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,DOB,DOD,EXPIRE_FLAG,AGE_AT_ADMISSION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,TEXT
0,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-10,2157-08-10 18:46:00,Nursing/other,Report,CSRU ADMISSION NOTE:\n\nPT WITH HISTORY OF [**...
1,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-11,2157-08-11 05:05:00,Nursing/other,Report,SHIFT UPDATE 7P-7A:\n\nNEURO: CONTINUES ON PR...
2,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-11,2157-08-11 18:09:00,Nursing/other,Report,NURSING PROGRESS NOTE:\n\nNEURO: WEANED OFF PR...
3,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-12,2157-08-12 00:37:00,Nursing/other,Report,"NEURO: ALERT AND ORIENTED X3, VERY TALKATIVE W..."
4,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-12,2157-08-12 04:53:00,Nursing/other,Report,"AFIB RATES 90-140'S, STABLE BP. AMIODARONE BOL..."
5,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-16,NaT,Discharge summary,Report,Admission Date: [**2157-8-10**] Dischar...
6,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-16,NaT,ECG,Report,Atrial fibrillation\nLeft axis deviation\nRBBB...
7,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-18,NaT,Discharge summary,Report,Admission Date: [**2157-8-8**] Discharg...
8,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-18,NaT,ECG,Report,Magnet induced A-V sequentially paced rhythm a...
9,100,153952,2157-08-10 07:15:00,2157-08-18 19:54:00,ELECTIVE,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT;...,F,2085-08-31,NaT,0,71,2157-08-18,NaT,ECG,Report,Sinus rhythm with atrial sensing and ventricul...


In [ ]:
df = expanded_admission_df.copy()

df['ADMITTIME']  = pd.to_datetime(df['ADMITTIME'], errors='coerce')
df['DISCHTIME']  = pd.to_datetime(df['DISCHTIME'], errors='coerce')
df['DOD']        = pd.to_datetime(df['DOD'], errors='coerce')

# 90-Day Readmission
df['90_DAY_READMISSION'] = 0
mask_90_day = (
    (df['NEXT_ADMITTIME'] - df['DISCHTIME']).notna() &
    ((df['NEXT_ADMITTIME'] - df['DISCHTIME']).dt.days <= 90) &
    ((df['NEXT_ADMITTIME'] - df['DISCHTIME']).dt.days >= 0)
)
df.loc[mask_90_day, '90_DAY_READMISSION'] = 1

# 1-Year Mortality
df['1_YEAR_MORTALITY'] = 0
mask = (
    df['DOD'].notna() & 
    ((df['DOD'] - df['ADMITTIME']).dt.days <= 365) & 
    ((df['DOD'] - df['ADMITTIME']).dt.days >= 0)
)
df.loc[mask, '1_YEAR_MORTALITY'] = 1

df = df[df['DEATHTIME'].isna()]


In [245]:
df[(df['1_YEAR_MORTALITY'] == 1) & (df['90_DAY_READMISSION'] == 1)]

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,...,DOD,EXPIRE_FLAG,AGE_AT_ADMISSION,CHARTDATE,CHARTTIME,CATEGORY,DESCRIPTION,TEXT,1_YEAR_MORTALITY,90_DAY_READMISSION
1159,1006,189081,2159-04-30 10:23:00,2159-05-28 17:30:00,EMERGENCY,RUSS,JEWISH,MARRIED,WHITE,RESPIRATORY DISTRESS,...,2159-08-29,1,73,2159-05-28,NaT,Discharge summary,Report,Admission Date: [**2159-4-30**] ...,1,1
1781,10088,169938,2107-01-04 11:59:00,2107-01-11 15:45:00,EMERGENCY,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),WHITE,SEPSIS;PNEUMONIA;TELEMETRY,...,2107-07-20,1,77,2107-01-11,NaT,Discharge summary,Report,Admission Date: [**2107-1-4**] D...,1,1
2783,10144,107460,2202-10-11 07:15:00,2202-10-20 16:00:00,ELECTIVE,ENGL,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\MITRAL VALVE REPLACEME...,...,2203-06-14,1,57,2202-10-20,NaT,Discharge summary,Report,Admission Date: [**2202-10-11**] ...,1,1
2817,10144,138337,2202-11-04 13:53:00,2202-11-16 10:45:00,EMERGENCY,ENGL,CATHOLIC,MARRIED,WHITE,HYPOTENSION,...,2203-06-14,1,57,2202-11-16,NaT,Discharge summary,Report,Admission Date: [**2202-11-4**] ...,1,1
3085,10152,155412,2189-03-29 03:31:00,2189-04-18 16:00:00,EMERGENCY,NaN,CATHOLIC,MARRIED,WHITE,FEMUR FRACTURE,...,2189-07-10,1,73,2189-04-18,NaT,ECG,Report,Atrial paced\nMultifocal ventricular premature...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199519,99756,154665,2191-06-15 17:53:00,2191-07-06 17:55:00,EMERGENCY,ENGL,NOT SPECIFIED,SINGLE,WHITE,ALTERED MENTAL STATUS,...,2191-12-23,1,51,2191-07-06,NaT,Discharge summary,Report,Admission Date: [**2191-6-15**] ...,1,1
1199923,99781,167791,2133-07-23 17:11:00,2133-07-31 13:12:00,EMERGENCY,ENGL,NOT SPECIFIED,SINGLE,WHITE,MOTOR VEHICLE ACCIDENT WITH UNSPECIFIED INJURIES,...,2133-12-13,1,72,2133-08-01,NaT,ECG,Report,Sinus rhythm and occasional atrial ectopy. Lo...,1,1
1200507,99822,146997,2197-05-30 12:21:00,2197-05-31 18:00:00,EMERGENCY,ENGL,CATHOLIC,MARRIED,WHITE,SQUAMOUS CELL CANCER,...,2197-07-08,1,61,2197-05-31,NaT,Discharge summary,Report,Admission Date: [**2197-5-30**] ...,1,1
1202124,99982,151454,2156-11-28 11:56:00,2156-12-08 13:45:00,EMERGENCY,ENGL,CATHOLIC,MARRIED,WHITE,TVR,...,2157-03-04,1,65,2156-12-08,NaT,Discharge summary,Report,Admission Date: [**2156-11-28**] ...,1,1


<img src="image/example.png" width="50%">

<img src="image/prompt.png" width="50%">

# Glaucoma

slo_fundus: slo fundus image

age: patient age

gender: Female (0), Male (1)

race: Asian (0), Black (1), White (2)

ethnicity: non-Hispanic (0), Hispanic (1), Unknown (-1)

language: English (0), Spanish (1), Other (2), Unknown (-1)

maritalstatus: Marriage or Partnered (0), Single (1), Divorced (2), Widoled (3), Legally Separated (4), Unknown (-1)

glaucoma: Non-Glaucoma (0) or Glaucoma (1)

In [261]:
import numpy as np

# Load the .npz file directly using its filename
data = np.load('/secure/xiaoyang/HarvardFairVLMed/Dataset/Training/data_00002.npz')

# List the keys (arrays) in the file
print(data.files)  # Expected output: ['slo_fundus', 'maritalstatus', 'ethnicity', 'language', 'gender', 'race', 'age', 'glaucoma']

# Access the 'slo_fundus' array
slo_fundus_array = data['age']
print(slo_fundus_array)

['slo_fundus', 'maritalstatus', 'ethnicity', 'language', 'gender', 'race', 'age', 'glaucoma']
53


In [270]:
for file in file_list:
    data = np.load(file_path)
    print(f"Shape: {data['slo_fundus'].shape}")


Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (664, 512)
Shape: (66

In [259]:
import numpy as np
import pandas as pd
gender_map = {
    0: "Female",
    1: "Male"
}

race_map = {
    0: "Asian",
    1: "Black",
    2: "White"
}

ethnicity_map = {
    0: "non-Hispanic",
    1: "Hispanic",
    -1: "Unknown"
}

language_map = {
    0: "English",
    1: "Spanish",
    2: "Other",
    -1: "Unknown"
}

maritalstatus_map = {
    0: "Marriage or Partnered",
    1: "Single",
    2: "Divorced",
    3: "Widowed",
    4: "Legally Separated",
    -1: "Unknown"
}

glaucoma_map = {
    0: "Non-Glaucoma",
    1: "Glaucoma"
}

In [266]:
import glob
file_list = glob.glob('/secure/xiaoyang/HarvardFairVLMed/Dataset/Test/*.npz')
len(file_list)

2000

In [265]:
df_list = []

for file_path in file_list:
    data = np.load(file_path)

    ms_str = maritalstatus_map[data["maritalstatus"].item()]      
    eth_str = ethnicity_map[data["ethnicity"].item()]
    lang_str = language_map[data["language"].item()]
    gen_str  = gender_map[data["gender"].item()]
    race_str = race_map[data["race"].item()]
    age_val  = data["age"].item()
    glau_str = glaucoma_map[data["glaucoma"].item()]

    slo_fundus_array = data["slo_fundus"]  # shape (664, 512)
    
    row_df = pd.DataFrame({
        "slo_fundus": [slo_fundus_array],
        "maritalstatus": [ms_str],
        "ethnicity": [eth_str],
        "language": [lang_str],
        "gender": [gen_str],
        "race": [race_str],
        "age": [age_val],
        "glaucoma": [glau_str]
    })

    df_list.append(row_df)

final_df = pd.concat(df_list, ignore_index=True)
print(final_df.head())
print(final_df.shape)

                                          slo_fundus          maritalstatus  \
0  [[21, 26, 29, 19, 25, 19, 32, 18, 20, 21, 29, ...                 Single   
1  [[15, 14, 11, 15, 13, 13, 13, 14, 11, 11, 12, ...  Marriage or Partnered   
2  [[23, 25, 26, 28, 32, 28, 30, 26, 30, 32, 29, ...  Marriage or Partnered   
3  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  Marriage or Partnered   
4  [[53, 53, 68, 53, 55, 53, 50, 54, 62, 55, 60, ...                 Single   

      ethnicity language  gender   race age      glaucoma  
0  non-Hispanic  English    Male  White  73      Glaucoma  
1  non-Hispanic  English  Female  White  58  Non-Glaucoma  
2  non-Hispanic  English    Male  White  29  Non-Glaucoma  
3  non-Hispanic  English  Female  White  59  Non-Glaucoma  
4      Hispanic  English  Female  White  50  Non-Glaucoma  
(2000, 8)


In [271]:
final_df.at[0, 'slo_fundus'].shape

(664, 512)